<a href="https://colab.research.google.com/github/AlexanderBelfort/MDPRKT/blob/main/TopicModellingStage2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run in python console
import nltk
nltk.download('stopwords')

# Run in terminal or command prompt
%pip install spacy
%pip install pyLDAvis

In [107]:
import re
import string
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [84]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import Dataset
df = pd.read_csv('output_list.txt', sep='\t')
#print(df.target_names.unique())
#df.head()

# testing

#print(df.columns)
# returns R - Researcher and P - Participant which are not great names for columns
# df.columns[0]
# needs renaming to CONTENT
col_name = df.columns[0]
df=df.rename(columns = {col_name:'content'})
# testing
# df.columns[0]
# returns content now

df.head()

,content,P = Participant
0,R – [surgeon] dictating with Mr [name]. Parti...,NaN
1,P - At night time would would would would woul...,NaN
2,R - You are feeling pain. Okay.,NaN
3,P - Sometimes when there is a pain it comes in...,NaN
4,R - Okay.,NaN


In [108]:
# DATA CLEANING STEP
#
#
# Convert to list
data = df.content.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', token) for token in data]

# Remove distracting single quotes
data = [re.sub("\'", "", token) for token in data]

# remove text in square brackets like [surgeon], [name]
data = [re.sub('\[.*?\]', '', token) for token in data]

# remove numbers
data = [re.sub('\w*\d\w*', '', token) for token in data]

# remove punctuation
# might not be necessary now / we can do it later
# data = [re.sub('[%s]' % re.escape(string.punctuation), '', token) for token in data]

# all to lower
data = [word.lower() for word in data]

# filter out short tokens like P and R and i
# neccessary?
#data = [re.sub(r'\b\w{1,1}\b', '', token) for token in data]


pprint(data[:5])

['r –  dictating with mr . participant study number is .. thanks very much for '
 'coming along today. the reason that you’ve come up is obviously your general '
 'practitioner has conducted an ultrasound scan and you’ve been found to have '
 'stones in the gallbladder. are you having any symptoms from that just now? '
 'are you having any major problems? are you getting a lot of pain?',
 'p - at night time would would would would would, about  o’clock,  o’clock, i '
 'am feeling the pain.',
 'r - you are feeling pain. okay.',
 'p - sometimes when there is a pain it comes in the night time.',
 'r - okay.']


In [109]:
# Let’s tokenize each sentence into a list of words
# removing punctuations and unnecessary characters altogether
# on this step

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:3])

[['dictating', 'with', 'mr', 'participant', 'study', 'number', 'is', 'thanks', 'very', 'much', 'for', 'coming', 'along', 'today', 'the', 'reason', 'that', 'you', 've', 'come', 'up', 'is', 'obviously', 'your', 'general', 'practitioner', 'has', 'conducted', 'an', 'ultrasound', 'scan', 'and', 'you', 've', 'been', 'found', 'to', 'have', 'stones', 'in', 'the', 'gallbladder', 'are', 'you', 'having', 'any', 'symptoms', 'from', 'that', 'just', 'now', 'are', 'you', 'having', 'any', 'major', 'problems', 'are', 'you', 'getting', 'lot', 'of', 'pain'], ['at', 'night', 'time', 'would', 'would', 'would', 'would', 'would', 'about', 'clock', 'clock', 'am', 'feeling', 'the', 'pain'], ['you', 'are', 'feeling', 'pain', 'okay']]


In [105]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

# this does not work so good as the words
# gallblader surgery
# laporscopic cholefcystemy
# or any of that kind do not really appear often
# maybe mentioned once




['dictating', 'with', 'mr', 'participant', 'study', 'number', 'is', 'thanks', 'very', 'much', 'for', 'coming', 'along', 'today', 'the', 'reason', 'that', 'you', 've', 'come', 'up', 'is', 'obviously', 'your', 'general', 'practitioner', 'has', 'conducted', 'an', 'ultrasound', 'scan', 'and', 'you', 've', 'been', 'found', 'to', 'have', 'stones', 'in', 'the', 'gallbladder', 'are', 'you', 'having', 'any', 'symptoms', 'from', 'that', 'just', 'now', 'are', 'you', 'having', 'any', 'major', 'problems', 'are', 'you', 'getting', 'lot', 'of', 'pain']


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [110]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [115]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

# # # # # # # # # 
## # # # # # 
##
#

[['dictate', 'study', 'number', 'thank', 'much', 'come', 'along', 'today', 'reason', 'come', 'obviously', 'general', 'practitioner', 'conduct', 'find', 'stone', 'gallbladder', 'symptom', 'major', 'problem', 'get', 'lot', 'pain']]


In [122]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

# words really are not repeated that often :/
# maximum of two repetitions in the first sentence
# that is why I added would 5 times at index 1
# print(corpus[1])

[(13, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 5)]


In [124]:
# testing if the word with id 25 is actually would
# id2word[26]
# out: would
# test passed
# # # # # # # # # # # # # # # # # # # #

In [127]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('along', 1),
  ('come', 2),
  ('conduct', 1),
  ('dictate', 1),
  ('find', 1),
  ('gallbladder', 1),
  ('general', 1),
  ('get', 1),
  ('lot', 1),
  ('major', 1),
  ('much', 1),
  ('number', 1),
  ('obviously', 1),
  ('pain', 1),
  ('practitioner', 1),
  ('problem', 1),
  ('reason', 1),
  ('stone', 1),
  ('study', 1),
  ('symptom', 1),
  ('thank', 1),
  ('today', 1)],
 [('pain', 1),
  ('clock', 2),
  ('feeling', 1),
  ('night', 1),
  ('time', 1),
  ('would', 5)]]

In [133]:
# # # # # # # # # # # # # # # # #
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [132]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"send" + 0.017*"back" + 0.017*"ask" + 0.016*"need" + '
  '0.014*"gallstone" + 0.014*"chance" + 0.014*"side" + 0.014*"work" + '
  '0.014*"would" + 0.014*"mean"'),
 (1,
  '0.072*"name" + 0.071*"sign" + 0.041*"get" + 0.040*"put" + 0.040*"initial" + '
  '0.040*"box" + 0.038*"point" + 0.036*"print" + 0.036*"apologise" + '
  '0.036*"capital"'),
 (2,
  '0.067*"thank" + 0.036*"stop" + 0.024*"may" + 0.021*"fine" + '
  '0.014*"information" + 0.014*"contact" + 0.014*"sometimes" + 0.013*"take" + '
  '0.012*"say" + 0.012*"study"'),
 (3,
  '0.033*"go" + 0.026*"would" + 0.022*"study" + 0.022*"surgery" + '
  '0.016*"gallbladder" + 0.016*"liver" + 0.016*"part" + 0.014*"need" + '
  '0.012*"keyhole" + 0.011*"say"'),
 (4,
  '0.033*"pain" + 0.033*"take" + 0.032*"would" + 0.020*"time" + '
  '0.016*"treatment" + 0.014*"ibuprofen" + 0.013*"operation" + 0.013*"quite" + '
  '0.013*"get" + 0.012*"patient"')]


In [134]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.210910947738692

Coherence Score:  0.6221580178722319


In [135]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.101895  0.088055       1        1  31.926214
4     -0.101193 -0.070550       2        1  31.403435
2     -0.000003  0.017670       3        1  15.809250
0     -0.068799 -0.033453       4        1  13.298560
1      0.271889 -0.001722       5        1   7.562542, topic_info=         Term       Freq      Total Category  logprob  loglift
20      thank  12.000000  12.000000  Default  30.0000  30.0000
340      name   7.000000   7.000000  Default  29.0000  29.0000
369      sign   6.000000   6.000000  Default  28.0000  28.0000
104      stop   9.000000   9.000000  Default  27.0000  27.0000
7         get  11.000000  11.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
15    problem   0.245644   5.639858   Topic5  -5.8020  -0.5518
245     small   0.170223   4.335269   Topic5  -6.1688  -0.6555
178  whenever   0.151907   1.791959   Topic5  -6.2826   0.1142
45       need   0.161767  10.016748   Topic5  -6.2197  -1.5439
86     moment   0.146032   2.496026   Topic5  -6.3220  -0.2567

[259 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
46        1  0.549000  absolutely
46        2  0.366000  absolutely
46        3  0.183000  absolutely
36        3  0.658282       admit
211       2  0.838410   advantage
...     ...       ...         ...
26        1  0.402186       would
26        2  0.491560       would
26        3  0.044687       would
26        4  0.089375       would
351       3  0.705412        year

[283 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 1, 2])